In [1]:
import numpy as np
import pandas as pd
import math
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize
from nltk.stem import PorterStemmer
porter = PorterStemmer()


In [2]:
def _singer_song_dic(data):
    '''Build singer-song dictionay
    
    Dictionary format:
    {singer_name: {song_name: lyrics}}
    '''
    
    singer_songs = {}
    
    for i in range(len(data)):
        singer = data.iloc[i]['artist']
        song = data.iloc[i]['song']
        lyric = data.iloc[i]['lyrics']
        
        
        if singer not in singer_songs:
            temp = {song: lyric}
            singer_songs[singer] = temp
        else:
            singer_songs[singer][song] = lyric
            
    return singer_songs

In [3]:
# This function is tested
def _update_inverted_index(name, lyrics, inverted_index):
    '''Create inverted index, count doc vector length

    Read contents form file, remove punctuation and stopwords to get terms.
    Count tf of this doc, then update inverted index.
    
    inverted_index
    '''
    
    indices = {}
    punctuation = re.compile(r'[^\w\s\']')
    
    ###
    # Count term frequency
    ###
    stop_words = set(stopwords.words('english'))
    lyrics_split = re.sub(punctuation, ' ', lyrics.lower()).split()
    
    for term in lyrics_split:
        if term in stop_words:
            continue
        elif term in indices:
            indices[term] += 1
        else:
            indices[term] = 1
    
    ###
    # Update inverted_index
    ###
    for term, frequency in indices.items():
        if term in inverted_index:
            posting = inverted_index[term]
            posting.append((name, frequency))
            inverted_index[term] = posting
        else:
            inverted_index[term] = [(name, frequency)]

In [4]:
def _compute_weight(tf, df, n_songs):
    
    idf = math.log(n_songs / df)

    tf_normalized = 1 + math.log(tf)

    weight = tf_normalized * idf
    return weight

In [5]:
def _build_song_vector(inverted_index, n_songs):
    '''Build term-weight vector for songs
    
    Compute the tf-idf weight, {term: weight}
    
    return:
        dic: a dictionary which format is {song: {term: weight}}
    '''
    
    song_vectors = {}
    
    for term, posting in inverted_index.items():
        df = len(posting)
        for pair in posting:
            song, tf = pair
            weight = _compute_weight(tf, df, n_songs)
            if song not in song_vectors:  # Create song vec
                song_vectors[song] = {term: weight}
            else:
                song_vectors[song][term] = weight  # Add new term into vec
                
    return song_vectors

In [6]:
def _get_top_songs(inverted_index, terms):
    '''Count rep terms' frequency, use that to pick rep songs
    
    Args:
        inverted_index: inverted index of this singer's songs
            format => {term: [(song, tf),]}
        terms: this singer's representive terms
    Return:
        top_songs(list): a list that contains the name of songs, order by score
    '''
    
    song_scores = {}
    for term in terms:
        posting = inverted_index[term]
        for song, _ in posting:
            if song in song_scores:
                song_scores[song] += 1
            else:
                song_scores[song] = 1
                
    top_songs = sorted(song_scores.items(), key=lambda kv: kv[1], reverse=True)
    top_songs = [name for name, _ in top_songs]
    return top_songs

In [7]:

def _find_rep_songs(songs, num_of_songs, num_of_terms):
    '''Find the most representive songs of this singer
    
    Choose songs by compare the scores which is sum the weight of each important term in that song.
    
    Args:
        songs(dic): a dictionary {song: lyrics}
        num_of_songs: number of representive songs you want to choose
        num_of_terms: number of important words we want to score the songs
        
    Return:
        rep_songs(dic): a dictionary {song: lyrics}, size will less or equal to num_of_songs
    '''
    
    inverted_index = {}
    
    n_songs = len(songs)
    if n_songs <= num_of_songs:  # Do not need to choose if not has enough songs
        return songs
    
    # Build inverted index
    for name, lyrics in songs.items():
        _update_inverted_index(name, lyrics, inverted_index)
        
    # build song vector
    song_vectors = _build_song_vector(inverted_index, n_songs)
    
    # sort by weight, get top words (put them into a set)
    selected_terms = set()
    
    for song, vector in song_vectors.items():
        sorted_v = sorted(vector.items(), key=lambda kv: kv[1], reverse=True)
        select_range = sorted_v[:num_of_terms + 1]
        for term, _ in select_range:
            selected_terms.add(term)
    
    # get score of each song
    song_sorted = _get_top_songs(inverted_index, selected_terms)
    selected_songs = song_sorted[:num_of_songs + 1]
    top_songs = {}
    for name, lyrics in songs.items():
        if name in selected_songs:
            top_songs[name] = lyrics
    
    return top_songs

In [8]:
def get_rep_songs(num_of_songs, num_of_terms, file_path):
    '''Get each singer's representive songs
    
    Args: 
        num_of_songs: number of songs you want to get from each singer
        file_path: path of input data file
    Return:
        dic: A dictionary which format is {singer, {song: lyrics}}
    '''
    
    # Read data from file
    data = pd.read_csv(file_path)
    singer_songs = _singer_song_dic(data)
    
    representive_songs = {}  # return dic
    for singer, songs in singer_songs.items():
        rep = _find_rep_songs(songs, num_of_songs, num_of_terms)
        representive_songs[singer] = rep
    
    with open('top_songs.pickle', 'wb') as handle:
        pickle.dump(representive_songs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    return representive_songs

In [9]:
# TESTing part
files = 'mylyrics00.csv'
rep_data = get_rep_songs(20, 50, files)

In [10]:
import pandas as pd

data = pd.read_csv('mylyrics00.csv')
singer = (list(rep_data.keys()))
print(singer)

lyrics = []
for i in range(len(singer)):
    sing = singer[i]
    temp = ""
    song = list(rep_data[sing].keys())
    for j in range(len(song)):
        temp = temp + " " + rep_data[sing][song[j]]
    lyrics.append(temp)
    

['eminem', 'ghostface-killah', '50-cent', 'dr-dre', 'drake', '2pac', 'beyonce-knowles', 'ariana-grande', 'chris-brown', 'glen-campbell', 'ciara', 'ed-sheeran', 'garth-brooks', 'billy-ray-cyrus', 'brad-paisley', 'bryan-adams', 'eddy-arnold', 'beatles', 'coldplay', 'the-calling', 'the-doors']


In [11]:
print(len(rep_data['eminem'].values()))

21


In [12]:
print(lyrics[0])

 [Intro: Eminem]
Welcome to Detroit
This is the BET, Shady 2.0 Cypher 2011
Myself, Slaughterhouse and Yelawolf
White Dawg get em
[Verse 1: Yelawolf]
Put these muthafuckas in a box and I send em away
Put em in a gray 'llac and pop the trunk
And throw em in the back, jack; ha, dig em a grave
Put a brick inside that Xerox when I print up a page
Moving keys I can relate, cause I live in the cage
I throw up the A, I take em to school
I give them a grade
An easy E for effort, that's WWA
White with an attitude
Alphabet soup is on my plate
All I got is Z's they sleeping on me
I can't get em awake
I spoon feed em a sound
In a room full of deceivers and clowns
Who believe in making it rain cause all they see is the clouds
And I watch from the couch of the VIP like a potato
With a bunch of meatheads, like fuck it
I'll just feed em a cow
Plenty of White boys to pick from this year
But before you pick a pepper, you better pick up your heater
Cause even Peter Piper could pick up a mic
But what it's 

In [13]:
print(len(lyrics[0]))

158806


In [14]:
porter.stem('going')

'go'

In [15]:
from nltk.tokenize.regexp import regexp_tokenize
from tqdm import tqdm_notebook

test_list = []
#for i in range(len(lyrics)):
for i in tqdm_notebook(lyrics):
    list_a=[]
    temp=str(i)
    temp=temp.lower()
    temp2 = ""
    #for j in range(len(temp)):
    for j in tqdm_notebook(temp.split(' ')):
        temp2 = temp2 + " " + porter.stem(j)
    list_a=regexp_tokenize(temp2, "[a-z]['a-z]*")
    test_list.append(list_a)

In [16]:
print(len(test_list))

21


In [17]:
print(test_list[0])

['intro', 'eminem', 'welcom', 'to', 'detroit', 'thi', 'is', 'the', 'bet', 'shadi', 'cypher', 'myself', 'slaughterhous', 'and', 'yelawolf', 'whit', 'dawg', 'get', 'em', 'vers', 'yelawolf', 'put', 'these', 'muthafucka', 'in', 'a', 'box', 'and', 'i', 'send', 'em', 'away', 'put', 'em', 'in', 'a', 'gray', 'llac', 'and', 'pop', 'the', 'trunk', 'and', 'throw', 'em', 'in', 'the', 'back', 'jack', 'ha', 'dig', 'em', 'a', 'grave', 'put', 'a', 'brick', 'insid', 'that', 'xerox', 'when', 'i', 'print', 'up', 'a', 'page', 'mov', 'key', 'i', 'can', 'relate', 'caus', 'i', 'live', 'in', 'the', 'cage', 'i', 'throw', 'up', 'the', 'a', 'i', 'take', 'em', 'to', 'school', 'i', 'give', 'them', 'a', 'grade', 'an', 'easi', 'e', 'for', 'effort', "that'", 'wwa', 'whit', 'with', 'an', 'attitude', 'alphabet', 'soup', 'is', 'on', 'my', 'plate', 'al', 'i', 'got', 'is', "z'", 'they', 'sleep', 'on', 'me', 'i', "can't", 'get', 'em', 'awake', 'i', 'spoon', 'feed', 'em', 'a', 'sound', 'in', 'a', 'room', 'full', 'of', 'dece

In [18]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
stopWords = list(set(stopwords.words('english')))
#print(stopWords)

for i in range(len(test_list)):
    temp = []
    #for j in range(len(lyrics[i])):
    for j in test_list[i]:
        #print(j)
        if j not in stopWords:
            temp.append(j)
            #print(temp)
    test_list[i] = temp

In [19]:
lyrics_term = []
for i in range(len(test_list)):
    temp = {}
    for j in range(len(test_list[i])):
        try:
            temp[test_list[i][j]] += 1
        except KeyError:
            temp[test_list[i][j]] = 1
    lyrics_term.append(temp)

In [20]:
all_term = []
for i in range(len(lyrics)):
    for j in range(len(test_list[i])):
        all_term.append(test_list[i][j])
all_term = set(all_term)
print(len(all_term))

13646


In [21]:
all_lyrics_term = []
for i in range(len(lyrics_term)):
    temp = {}
    for j in all_term:
        try:
            temp[j] = lyrics_term[i][j]
        except KeyError:
            temp[j] = 0
    all_lyrics_term.append(temp)

In [22]:
a = {}
for i in range(len(singer)):
    a[singer[i]] = all_lyrics_term[i]

In [23]:
df = pd.DataFrame(a, columns=singer)

In [24]:
df

,eminem,ghostface-killah,50-cent,dr-dre,drake,2pac,beyonce-knowles,ariana-grande,chris-brown,glen-campbell,...,ed-sheeran,garth-brooks,billy-ray-cyrus,brad-paisley,bryan-adams,eddy-arnold,beatles,coldplay,the-calling,the-doors
a',0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
a'rattlin',0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
a'ringing,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
aaaaah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
aaaallll,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,1
aahm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ab,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aback,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
term = list(df.T.columns)

In [26]:
a = df.T
original = a.copy()
a

,a',a'rattlin',a'ringing,aa,aaaaah,aaaallll,aah,aahm,ab,aback,...,zod,zoe,zombi,zone,zones,zoo,zoom,zow,zulu,zz
eminem,0,0,0,0,0,0,0,0,2,0,...,2,0,2,0,0,0,3,0,0,0
ghostface-killah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,11,1,0,1,0
50-cent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dr-dre,0,0,0,0,0,0,3,0,0,0,...,0,0,0,1,0,0,0,0,0,0
drake,0,0,0,0,0,1,0,0,0,1,...,0,1,0,2,0,0,0,0,0,1
2pac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,1,0,0
beyonce-knowles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ariana-grande,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chris-brown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
glen-campbell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
print(term)

["a'", "a'rattlin'", "a'ringing", 'aa', 'aaaaah', 'aaaallll', 'aah', 'aahm', 'ab', 'aback', 'abandon', 'abbott', 'abc', 'abcdefghijklmnopqrstuvwxy', 'abdomen', 'abe', 'abid', 'abil', 'ability', 'abl', 'able', 'aboard', 'aborn', 'abort', 'aborted', 'abortion', 'abortions', 'abounding', 'abov', 'abraham', 'abroad', 'absenc', 'absence', 'absinth', 'absinthe', 'absolut', 'absolutely', 'absorb', 'absorbs', 'absorption', 'absurd', 'abundance', 'abus', 'abusers', 'ac', 'academy', 'acapella', 'accent', 'accept', 'accid', 'accident', 'accolad', 'accomplishments', 'accord', "accordin'", 'account', "account'", 'accountants', 'accounts', 'accused', 'ace', 'ach', 'ache', 'achiev', 'achieve', 'acid', 'acknowledg', 'acknowledged', 'acorn', 'acoust', 'acquir', 'acquired', 'acquitted', 'acr', 'acrobat', 'across', 'act', 'actavi', 'actin', "actin'", 'action', 'actions', 'activ', 'activated', 'activist', 'activit', 'actor', 'actors', 'actress', 'actual', 'acura', 'ad', 'ada', 'adam', 'adams', 'adamsville

In [28]:
from tqdm import tqdm_notebook
count2 = 0
for i in tqdm_notebook(term):
    count = 0
   
    for j in range(len(singer)):
        if a.iloc[j][i] != 0:
            count += 1
    if count > (len(singer) / 3):
        count2 += 1
        #a.drop_duplicates(i, keep=False)
        a = a.drop(i, axis=1)
print(count2)


1123


In [29]:
a

,a',a'rattlin',a'ringing,aa,aaaaah,aaaallll,aah,aahm,ab,aback,...,zl,zod,zoe,zombi,zones,zoo,zoom,zow,zulu,zz
eminem,0,0,0,0,0,0,0,0,2,0,...,17,2,0,2,0,0,3,0,0,0
ghostface-killah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,11,1,0,1,0
50-cent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dr-dre,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
drake,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2pac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
beyonce-knowles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ariana-grande,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chris-brown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
glen-campbell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return round(dot(A, B)/(norm(A)*norm(B)), 3)

In [31]:
test = a.copy()
a = a.T
a
#word_list=list(a.columns)
word_list=list(test.columns)

In [32]:
from operator import itemgetter
first_singer=[]
second_singer = []
for i in range(len(test.iloc[0])):
    first_singer.append([test.iloc[0][i],word_list[i]])
    second_singer.append([test.iloc[1][i],word_list[i]])
    
first_singer.sort(key=itemgetter(0),reverse=True)
second_singer.sort(key=itemgetter(0),reverse=True)

#b = np.asarray(a.iloc[:][singer[0]])
#c = np.asarray(a.iloc[:][singer[1]])

In [33]:
print(first_singer)

[[51, 'eminem'], [21, 'album'], [21, 'detroit'], [21, 'zan'], [20, 'faggot'], [19, 'lowe'], [18, 'statik'], [18, 'vegas'], [17, 'abl'], [17, 'zl'], [16, 'slim'], [15, 'attack'], [14, 'idea'], [13, 'proof'], [12, 'evil'], [12, 'nod'], [12, 'urm'], [12, 'vs'], [11, 'ayo'], [11, 'mc'], [11, 'twin'], [10, 'also'], [10, 'gay'], [10, 'lp'], [10, 'shadi'], [10, 'shady'], [10, 'shock'], [10, 'welcom'], [9, 'actual'], [9, 'pac'], [9, 'rick'], [9, 'wack'], [9, 'whitey'], [8, 'aint'], [8, 'boys'], [8, 'ck'], [8, 'closet'], [8, 'dre'], [8, 'everybody'], [8, 'figur'], [8, "here'"], [8, 'ill'], [8, 'marshal'], [8, 'monster'], [8, 'piss'], [8, 'pit'], [8, 'rihanna'], [8, 'salut'], [8, 'slaughterhous'], [8, 'stupid'], [8, 'yelawolf'], [7, 'addict'], [7, 'beef'], [7, 'budden'], [7, 'buri'], [7, 'certain'], [7, 'cking'], [7, 'cock'], [7, 'consid'], [7, 'direct'], [7, 'fag'], [7, 'greatest'], [7, 'guitar'], [7, 'mather'], [7, 'mental'], [7, 'ortiz'], [7, 'pill'], [7, 'royc'], [7, 'sack'], [7, 'shut'], [7

In [34]:
print(second_singer)

[[32, 'ghostfac'], [27, 'killah'], [27, 'wu'], [18, 'celebrate'], [18, 'raekwon'], [17, 'niggaz'], [16, 'island'], [12, 'ghostface'], [11, 'bone'], [11, 'ill'], [11, 'raw'], [11, 'zoo'], [10, 'plu'], [10, 'smash'], [10, 'stash'], [9, 'cappadonna'], [9, 'iron'], [9, 'tang'], [9, 'tone'], [8, 'aiyo'], [8, 'grill'], [8, 'punk'], [8, 'rat'], [8, 'snatch'], [7, 'intro'], [7, 'killa'], [7, 'relax'], [7, 'shaolin'], [7, 'staten'], [7, 'toast'], [7, 'trife'], [6, 'armor'], [6, 'belt'], [6, 'brooklyn'], [6, 'bulletproof'], [6, 'champ'], [6, 'chef'], [6, "don'"], [6, 'mc'], [6, 'na'], [6, 'rob'], [6, 'rza'], [6, 'shock'], [6, 'theodore'], [6, 'violenc'], [5, 'blunt'], [5, 'cappa'], [5, 'dash'], [5, 'doubl'], [5, 'drunk'], [5, 'especi'], [5, 'faggot'], [5, 'frank'], [5, 'ga'], [5, 'gat'], [5, 'golden'], [5, 'gray'], [5, "here'"], [5, 'jeans'], [5, "lil'"], [5, 'master'], [5, 'mov'], [5, 'odb'], [5, 'piss'], [5, 'project'], [5, 'sho'], [5, 'snap'], [5, 'snitch'], [5, 'stretch'], [5, 'tap'], [5, 't

In [35]:
aaa = []
for i in range(len(singer)):
    aaa.append(np.asarray(a.iloc[:][singer[i]]))

# remove 1/2

In [36]:
cos_sim_a=[]

for i in range(len(singer)):
    list_sim=[]
    for j in range(len(singer)):
        list_sim.append(cos_sim(aaa[i],aaa[j]))
  
    cos_sim_a.append(list_sim)
for i in range(len(singer)):
    print(cos_sim_a[i])

[1.0, 0.179, 0.17, 0.15, 0.123, 0.129, 0.06, 0.067, 0.095, 0.056, 0.048, 0.1, 0.074, 0.064, 0.08, 0.036, 0.059, 0.055, 0.059, 0.05, 0.071]
[0.179, 1.0, 0.205, 0.151, 0.073, 0.188, 0.067, 0.025, 0.087, 0.052, 0.063, 0.079, 0.053, 0.049, 0.065, 0.078, 0.049, 0.048, 0.052, 0.028, 0.076]
[0.17, 0.205, 1.0, 0.179, 0.089, 0.28, 0.056, 0.033, 0.08, 0.04, 0.044, 0.058, 0.04, 0.056, 0.05, 0.09, 0.043, 0.023, 0.029, 0.028, 0.048]
[0.15, 0.151, 0.179, 1.0, 0.058, 0.21, 0.056, 0.019, 0.054, 0.015, 0.042, 0.035, 0.039, 0.023, 0.027, 0.075, 0.026, 0.038, 0.032, 0.014, 0.03]
[0.123, 0.073, 0.089, 0.058, 1.0, 0.08, 0.039, 0.044, 0.099, 0.035, 0.044, 0.058, 0.028, 0.023, 0.038, 0.033, 0.021, 0.035, 0.019, 0.015, 0.025]
[0.129, 0.188, 0.28, 0.21, 0.08, 1.0, 0.04, 0.032, 0.056, 0.046, 0.042, 0.049, 0.045, 0.054, 0.031, 0.154, 0.045, 0.028, 0.023, 0.019, 0.07]
[0.06, 0.067, 0.056, 0.056, 0.039, 0.04, 1.0, 0.055, 0.042, 0.014, 0.04, 0.034, 0.021, 0.027, 0.022, 0.023, 0.015, 0.015, 0.028, 0.009, 0.034]
[0.0

# original(don't remove)

In [37]:
original = original.T
original_word_list=list(original.columns)

bbb = []
for i in range(len(singer)):
    bbb.append(np.asarray(original.iloc[:][singer[i]]))
cos_sim_b=[]

for i in range(len(singer)):
    list_sim=[]
    for j in range(len(singer)):
        list_sim.append(cos_sim(bbb[i],bbb[j]))
  
    cos_sim_b.append(list_sim)
for i in range(len(singer)):
    print(cos_sim_b[i])

[1.0, 0.782, 0.786, 0.807, 0.829, 0.704, 0.717, 0.531, 0.812, 0.584, 0.668, 0.685, 0.545, 0.616, 0.728, 0.546, 0.519, 0.525, 0.542, 0.621, 0.461]
[0.782, 1.0, 0.794, 0.792, 0.742, 0.719, 0.638, 0.476, 0.775, 0.516, 0.608, 0.51, 0.485, 0.528, 0.617, 0.497, 0.443, 0.458, 0.449, 0.491, 0.485]
[0.786, 0.794, 1.0, 0.821, 0.766, 0.785, 0.618, 0.478, 0.808, 0.463, 0.614, 0.534, 0.432, 0.499, 0.595, 0.497, 0.414, 0.407, 0.469, 0.526, 0.431]
[0.807, 0.792, 0.821, 1.0, 0.758, 0.767, 0.636, 0.47, 0.771, 0.487, 0.598, 0.552, 0.48, 0.524, 0.625, 0.509, 0.441, 0.436, 0.459, 0.543, 0.458]
[0.829, 0.742, 0.766, 0.758, 1.0, 0.693, 0.647, 0.505, 0.788, 0.537, 0.627, 0.604, 0.469, 0.54, 0.66, 0.498, 0.457, 0.46, 0.479, 0.581, 0.461]
[0.704, 0.719, 0.785, 0.767, 0.693, 1.0, 0.577, 0.442, 0.698, 0.447, 0.549, 0.476, 0.439, 0.48, 0.561, 0.487, 0.405, 0.378, 0.44, 0.485, 0.445]
[0.717, 0.638, 0.618, 0.636, 0.647, 0.577, 1.0, 0.559, 0.703, 0.538, 0.69, 0.563, 0.469, 0.535, 0.667, 0.596, 0.409, 0.499, 0.525, 0

In [38]:
print(singer)

['eminem', 'ghostface-killah', '50-cent', 'dr-dre', 'drake', '2pac', 'beyonce-knowles', 'ariana-grande', 'chris-brown', 'glen-campbell', 'ciara', 'ed-sheeran', 'garth-brooks', 'billy-ray-cyrus', 'brad-paisley', 'bryan-adams', 'eddy-arnold', 'beatles', 'coldplay', 'the-calling', 'the-doors']


In [39]:
print(rep_data.keys())

dict_keys(['eminem', 'ghostface-killah', '50-cent', 'dr-dre', 'drake', '2pac', 'beyonce-knowles', 'ariana-grande', 'chris-brown', 'glen-campbell', 'ciara', 'ed-sheeran', 'garth-brooks', 'billy-ray-cyrus', 'brad-paisley', 'bryan-adams', 'eddy-arnold', 'beatles', 'coldplay', 'the-calling', 'the-doors'])


# cosine similarity with original data

In [41]:
for i in range(len(test)):
    test = cos_sim_b[i][:]
    test.pop(test.index(max(test)))
    a = cos_sim_b[i].index(max(test))
    print("" + singer[i] +"      "  + singer[a]+ " :   " + str(max(test)))

eminem      drake :   0.829
ghostface-killah      50-cent :   0.794
50-cent      dr-dre :   0.821
dr-dre      50-cent :   0.821
drake      eminem :   0.829
2pac      50-cent :   0.785
beyonce-knowles      eminem :   0.717
ariana-grande      ciara :   0.62
chris-brown      eminem :   0.812
glen-campbell      brad-paisley :   0.671
ciara      beyonce-knowles :   0.69
ed-sheeran      eminem :   0.685
garth-brooks      brad-paisley :   0.638
billy-ray-cyrus      brad-paisley :   0.694
brad-paisley      eminem :   0.728
bryan-adams      brad-paisley :   0.613
eddy-arnold      billy-ray-cyrus :   0.656
beatles      brad-paisley :   0.619
coldplay      the-calling :   0.605
the-calling      brad-paisley :   0.717


In [42]:
for i in range(len(cos_sim_b)):
    a = cos_sim_b[i].index(min(cos_sim_b[i]))
    print("" + singer[i] +"/"  + singer[a]+ " :   " + str(min(cos_sim_b[i])))

eminem/the-doors :   0.461
ghostface-killah/eddy-arnold :   0.443
50-cent/beatles :   0.407
dr-dre/beatles :   0.436
drake/eddy-arnold :   0.457
2pac/beatles :   0.378
beyonce-knowles/eddy-arnold :   0.409
ariana-grande/the-doors :   0.391
chris-brown/eddy-arnold :   0.389
glen-campbell/2pac :   0.447
ciara/eddy-arnold :   0.429
ed-sheeran/the-doors :   0.373
garth-brooks/ed-sheeran :   0.409
billy-ray-cyrus/2pac :   0.48
brad-paisley/the-doors :   0.552
bryan-adams/eddy-arnold :   0.427
eddy-arnold/chris-brown :   0.389
beatles/2pac :   0.378
coldplay/ariana-grande :   0.411
the-calling/2pac :   0.485
the-doors/ed-sheeran :   0.373


# cosine similarity between removed data

In [43]:
for i in range(len(test)):
    test = cos_sim_a[i][:]
    test.pop(test.index(max(test)))
    a = cos_sim_a[i].index(max(test))
    print("" + singer[i] +"      "  + singer[a]+ " :   " + str(max(test)))

eminem      ghostface-killah :   0.179
ghostface-killah      50-cent :   0.205
50-cent      2pac :   0.28
dr-dre      2pac :   0.21
drake      eminem :   0.123
2pac      50-cent :   0.28
beyonce-knowles      ghostface-killah :   0.067
ariana-grande      ed-sheeran :   0.099
chris-brown      brad-paisley :   0.108
glen-campbell      eddy-arnold :   0.1
ciara      ghostface-killah :   0.063
ed-sheeran      eminem :   0.1
garth-brooks      billy-ray-cyrus :   0.11
billy-ray-cyrus      garth-brooks :   0.11
brad-paisley      chris-brown :   0.108
bryan-adams      2pac :   0.154
eddy-arnold      glen-campbell :   0.1
beatles      eminem :   0.055
coldplay      ed-sheeran :   0.072
the-calling      glen-campbell :   0.092


In [44]:
for i in range(len(cos_sim_a)):
    a = cos_sim_a[i].index(min(cos_sim_a[i]))
    print("" + singer[i] +"/"  + singer[a]+ " :   " + str(min(cos_sim_a[i])))

eminem/bryan-adams :   0.036
ghostface-killah/ariana-grande :   0.025
50-cent/beatles :   0.023
dr-dre/the-calling :   0.014
drake/the-calling :   0.015
2pac/the-calling :   0.019
beyonce-knowles/the-calling :   0.009
ariana-grande/eddy-arnold :   0.015
chris-brown/glen-campbell :   0.012
glen-campbell/ciara :   0.006
ciara/glen-campbell :   0.006
ed-sheeran/chris-brown :   0.027
garth-brooks/chris-brown :   0.016
billy-ray-cyrus/ciara :   0.014
brad-paisley/ariana-grande :   0.018
bryan-adams/ciara :   0.015
eddy-arnold/ciara :   0.012
beatles/coldplay :   0.014
coldplay/beatles :   0.014
the-calling/ciara :   0.007
the-doors/ciara :   0.018
